In [3]:
f = open("C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/HaeinAPIKey.txt", "r")
API_Key = f.readline()
API_Key = API_Key.strip("\n")
decoded_API_Key =f.readline()
f.close()

In [4]:
API_Key = decoded_API_Key

In [5]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [4]:
def sigunguDF(data):
    item_list = data.find_all('item')
    data_list = []
    for item in item_list:
        if(item.find("kaptName")!=None):
            Complex_Name = item.find("kaptName").get_text()
        else:
            Complex_Name = item.find("kaptname").get_text()
        
        if(item.find("kaptCode")!=None):
            kaptCode = item.find("kaptCode").get_text()
        else:
            kaptCode = item.find("kaptcode").get_text()

        if(item.find("bjdCode")!=None):
            bjdCode = item.find("bjdCode").get_text()
        else:
            bjdCode = item.find("bjdcode").get_text()

        data_dict = {
                "Complex_Name": Complex_Name,
                "kaptCode": kaptCode,
                "bjdCode": bjdCode,
            }
        data_list.append(data_dict)

    df = pd.DataFrame(data_list)
    return df

In [5]:
def getSigunguAptList(API_Key, code):
    """
    Input sigunguCode to get list of Apartment complexes and their codes
    code -- sigunguCode input; Code of the region given by the government
    """
    url = "http://apis.data.go.kr/1613000/AptListService2/getSigunguAptList"
    params ={'serviceKey' : API_Key , 
             'sigunguCode' : code, 
             'pageNo': 1,
             }

    response = requests.get(url, params=params)
    first_data = BeautifulSoup(response.text, "xml")

    df_init = sigunguDF(first_data)
    if(first_data.find('totalCount')!=None):
        total = int(first_data.find('totalCount').get_text())
    else:
        total = int(first_data.find('totalcount').get_text())

    totPage = (total // 10) + 2
    
    for page in range(2,totPage):    
        url = "http://apis.data.go.kr/1613000/AptListService2/getSigunguAptList"
        params ={'serviceKey' : API_Key , 
                'sigunguCode' : code, 
                'pageNo': page}

        response = requests.get(url, params=params)
        data = BeautifulSoup(response.text, "xml")
        df = sigunguDF(data)
        df_init = pd.concat([df_init, df], ignore_index=True)
        
    return df_init

In [6]:
def AddInfo(df):
    AptList = []
    for Apt in df['kaptCode']:
        url = "http://apis.data.go.kr/1613000/AptBasisInfoService1/getAphusBassInfo"
        params ={'serviceKey' : API_Key , 
            'kaptCode' : Apt, 
            }
        response = requests.get(url, params=params)
        data = BeautifulSoup(response.text, "xml")
        if(data.find('kaptdaCnt') != None):
            Household_Num = data.find('kaptdaCnt').get_text()
        elif(data.find('kaptdacnt')!=None): 
            Household_Num = data.find('kaptdacnt').get_text()
        else: Household_Num = np.nan

        if(data.find('kaptUsedate')!=None):
            Begin_Date = data.find('kaptUsedate').get_text()
        elif(data.find('kaptusedate')!=None):
            Begin_Date = data.find('kaptusedate').get_text()
        else: Begin_Date = np.nan

        if(data.find('codeHeatNm')!=None):
            Heating_Method = data.find('codeHeatNm').get_text()
        elif(data.find('codeheatnm')!=None): 
            Heating_Method = data.find('codeheatnm').get_text()
        else:
            Heating_Method = np.nan

        if(data.find('codeAptNm')!=None):
            House_Type = data.find('codeAptNm').get_text()
        elif(data.find('codeaptnm')!=None): 
            House_Type = data.find('codeaptnm').get_text()
        else:
            House_Type = np.nan

        data_dict = {
        "kaptCode": Apt,
        "Household_Num":Household_Num,
        "Begin_Date":Begin_Date,
        "Heating_Method":Heating_Method,
        "House_Type":House_Type,
        }
        AptList.append(data_dict)

    df_2 = pd.DataFrame(AptList)
    df = df.merge(df_2, on= "kaptCode")
    return df


In [8]:
def cityIn(sigunguCode, API_Key, GuName):
    data =  getSigunguAptList(API_Key, sigunguCode)
    data = AddInfo(data)
    display(data)
    data.to_csv(f"C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/AptList/{GuName}.csv")

In [9]:
cityIn(11305, API_Key, "Gangbuk_Gu")

,Complex_Name,kaptCode,bjdCode,Household_Num,Begin_Date,Heating_Method,House_Type
0,미아동부센트레빌,A14280502,1130510100,480,20060425,개별난방,아파트
1,SK북한산시티아파트,A14272304,1130510100,3830,20040507,개별난방,아파트
2,한일유앤아이,A14272303,1130510100,384,20021129,개별난방,아파트
3,미아2동경남아너스빌,A14282301,1130510100,208,20080125,개별난방,아파트
4,삼각산아이원임대,A14210001,1130510100,673,20031230,개별난방,아파트
5,SK북한산시티임대,A14272311,1130510100,1497,20040507,개별난방,아파트
6,미아경남아너스빌,A14272306,1130510100,860,20030620,개별난방,아파트
7,래미안트리베라2차,A14272308,1130510100,1330,20100528,개별난방,아파트
8,벽산라이브파크,A14272305,1130510100,1585,20040807,개별난방,아파트
9,송천센트레빌,A14272313,1130510100,376,20100910,개별난방,아파트


In [6]:
guDict =  {"Dobong": 11320,
         "Dongdaemun": 11230, 
         "Dongjak": 11590, 
         "Eunpyeong":11380,
         "Gangbuk": 11305,
         "Gangdong": 11740,
         "Gangnam": 11680,
         "Gangseo":11500,
         "Geumcheon": 11545,
         "Guro": 11530,
         "Gwanak": 11620,
         "Gwangjin": 11215,
         "Jongno":11110,
         "Jung":11140,
         "Jungnang":11260,
         "Mapo":11440,
         "Nowon":11350,
         "Seocho":11650,
         "Seodaemun":11410,
         "Seongbuk":11290,
         "Seongdong":11200,
         "Songpa":11710,
         "Yangcheon":11470,
         "Yeongdeungpo":11560,
         "Yongsan":11170}

In [7]:
def getElecVar(API_Key, dfApt):
    AptList = []
    for Apt in dfApt:
        url = "http://apis.data.go.kr/1613000/AptBasisInfoService1/getAphusDtlInfo"
        params ={'serviceKey' : API_Key , 
            'kaptCode' : Apt, 
            }
        response = requests.get(url, params=params)
        data = BeautifulSoup(response.text, "xml")
        if(data.find('codeEcon') != None):
            Elec_Con = data.find('codeEcon').get_text()
        elif(data.find('codeecon')!=None): 
            Elec_Con = data.find('codeecon').get_text()
        else: Elec_Con = np.nan

        if(data.find('kaptdEcnt')!=None):
            Elev_Num = data.find('kaptdEcnt').get_text()
        elif(data.find('kaptdecnt')!=None):
            Elev_Num = data.find('kaptdecnt').get_text()
        else: Elev_Num = np.nan

        if(data.find('kaptdPcnt')!=None):
            Park_Above = data.find('kaptdPcnt').get_text()
        elif(data.find('kaptdpcnt')!=None): 
            Park_Above = data.find('kaptdpcnt').get_text()
        else:
            Park_Above = np.nan

        if(data.find('kaptdPcntu')!=None):
            Park_Below = data.find('kaptdPcntu').get_text()
        elif(data.find('kaptdpcntu')!=None): 
            Park_Below = data.find('kaptdpcntu').get_text()
        else:
            Park_Below = np.nan

        data_dict = {
        "kaptCode": Apt,
        "Elec_Con":Elec_Con,
        "Elev_Num":Elev_Num,
        "Park_Above":Park_Above,
        "Park_Below":Park_Below,
        }
        AptList.append(data_dict)

    df_2 = pd.DataFrame(AptList)
    return df_2

In [20]:
def AddMore(guDict, API_Key):
    for GuName in guDict.keys():
        df = pd.read_csv(f"C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/AptList/{GuName}_Gu.csv", index_col = 0)
        dfApt = df['kaptCode']
        df_2 = getElecVar(API_Key, dfApt)
        df = df.merge(df_2, on= "kaptCode")
        display(df)
        df.to_csv(f"C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/AptList/{GuName}_more.csv", index = False)                

In [ ]:
AddMore(guDict, API_Key)

In [8]:
GuName = 'Seongbuk'
df = pd.read_csv(f"C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/AptList/{GuName}_Gu.csv", index_col = 0)
dfApt = df['kaptCode']
df_2 = getElecVar(API_Key, dfApt)
df = df.merge(df_2, on= "kaptCode")
display(df)
df.to_csv(f"C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/AptList/{GuName}_more.csv", index = False)           

,Complex_Name,kaptCode,bjdCode,Household_Num,Begin_Date,Heating_Method,House_Type,Elec_Con,Elev_Num,Park_Above,Park_Below
0,돈암신일해피트리,A13679903,1129010300,192,20130115,개별난방,아파트,단일계약,12,0,319
1,돈암포스코더샵,A13606002,1129010300,339,20060331,개별난방,아파트,종합계약,11,93,290
2,돈암풍림,A13606101,1129010300,460,19990511,개별난방,아파트,단일계약,14,130,441
3,돈암일신건영휴먼빌,A13606003,1129010300,689,20050601,개별난방,아파트,단일계약,22,288,691
4,돈암삼성임대,A13606106,1129010300,736,19990430,개별난방,아파트,단일계약,8,26,139
...,...,...,...,...,...,...,...,...,...,...,...
129,석관중앙하이츠,A13681701,1129013900,315,19940624,개별난방,아파트,단일계약,7,62,179
130,래미안석관,A13676101,1129013900,580,20090626,개별난방,아파트,종합계약,19,0,684
131,석관래미안제2임대,A13676102,1129013900,80,20090626,개별난방,아파트,단일계약,2,0,56
132,석관코오롱,A13615002,1129013900,453,19990612,개별난방,아파트,단일계약,10,104,357


In [9]:
df = pd.read_csv(f"C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/AptList/{GuName}_more.csv", 
                         index_col = 0)
df = df[df['House_Type'] == '아파트']

        
df.to_csv(f"C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/AptOnly/{GuName}_Gu_Only.csv", index = False)

In [10]:
df2 = pd.read_csv("C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/AptOnly/Seoul_Only.csv")
df2 = pd.concat([df2, df])
df2.to_csv("C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/AptOnly/Seoul_Only.csv")

In [43]:
dfList = []
for GuName in guDict.keys():
    df = pd.read_csv(f"C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/AptList/{GuName}_more.csv", 
                         index_col = 0)
    df = df[df['House_Type'] == '아파트']
    dfList.append(df)
        
    df.to_csv(f"C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/AptOnly/{GuName}_Gu_Only.csv", index = False)

df2 = pd.concat(dfList)
        

In [45]:
df2.to_csv("C:/Users/lhi30/Haein/2023/YBIGTA/DA/Project/Share/AptOnly/Seoul_Only.csv")
